In [1]:
# Parameterization
#
# Often times notebooks will need to execute by injecting variables and values from externally to the notebook. This might be service credentials, specific
# endpoints, variations for the notebook (e.g. if a notebook is executing a hyperparameter grid search, they may be
# executed many times in parallel but only with external variables changing). Today, there are two primary ways that notebooks are parameterized - via new tool
# (like Papermill) or with code generation directly into the notebook (e.g. regex replacement against fields
# below). 
#
# Today: Values are just hard coded in, and require manual changes

epochs = 200
data_source = "http://data.contoso.com/blob/important_data.csv"
postgresql_credentials = ""

In [2]:
# Lack Environment Description
# 
# Import behavior is always bad, but most commonly, people will import several packages at the top of a file. They are unlikely to include specific versions
# and may use structures which are hard to introspect (e.g. 'from foo import bar as qaz'). However, the libraries may be imported outside the notebook itself, via 
# inline bash commands or via a command line (Jupyter notebooks execute inside the command line environment - so if packages were imported there, the notebook will
# run normally). This will often lead to mismatched environments when the package is deployed to another environment or containerized. Because the time in
# deployment of complex pipelines is so long, this could be 10 minutes or more before noticing that something is wrong.

# Requires the packages are already installed (lack version)
import numpy
import matplotlib


# Inline bash (bad) installs a package which may be different than the running kernel
!pip install tensorflow

# 
# Includes local bash
# No exception handling between cells
# Cannot be executed headlessly
# No caching for common functions
# No parallel execution
# Missed opportunity to run step statelessly
# High Mem
# High GPU
# Out of order execution (to prevent blocking)
# No retry automation for external service
# No parameter-ization


     |████████████████████████████████| 454.4 MB 50 kB/s 
     |████████████████████████████████| 1.2 MB 67.2 MB/s 
     |████████████████████████████████| 4.4 MB 72.0 MB/s 
     |████████████████████████████████| 57 kB 2.6 MB/s 
     |████████████████████████████████| 6.0 MB 39.0 MB/s 
     |████████████████████████████████| 1.0 MB 87.8 MB/s 
     |████████████████████████████████| 14.9 MB 67.3 MB/s 
     |████████████████████████████████| 129 kB 62.9 MB/s 
     |████████████████████████████████| 65 kB 1.8 MB/s 
     |████████████████████████████████| 42 kB 549 kB/s 
     |████████████████████████████████| 462 kB 94.4 MB/s 
     |████████████████████████████████| 4.0 MB 106.2 MB/s 
     |████████████████████████████████| 4.9 MB 67.4 MB/s 
     |████████████████████████████████| 288 kB 82.6 MB/s 
     |████████████████████████████████| 61 kB 3.8 MB/s 
     |████████████████████████████████| 146 kB 59.1 MB/s 
     |████████████████████████████████| 781 kB 98.1 MB/s 
     |██████████████